TO DO:
- [ ] WORK OUT % OF TOTAL ENERGY CONSUMPTION / VARIANCE THAT CAN BE EXPLAINED BY TOTAL CONFIGS CHANGES VS BASELINE THAT IS MODEL DETERMINED
- [ ] set up hoerarchical model -> identify groupings:
   - [ ] use model (even though currently only one model)
   - [ ] use config_name and date_time
   - [ ] use decoder configs
   - [ ] latency configs 

In [1]:
from scripts.a_data_loading_cleaning import run_load_clean_diagnose_data

csv_path = f"results/controlled_results.csv"
df = run_load_clean_diagnose_data(csv_path)

✅ Total generated tokens value is constant: 16384
Original distribution:
total_generated_tokens
16384    2682
Name: count, dtype: int64
----------------------------------------------------------------------------------------------------
Round 1: Verfifying FLOPs on raw df
NB: FLOPs values are NOT constant: [1.69499710e+13 2.02486233e+13 0.00000000e+00 5.26385823e+13]
Original distribution:
flops
0.000000e+00       5
1.694997e+13      22
2.024862e+13    2265
5.263858e+13     390
Name: count, dtype: int64

Dominant FLOPs value: 20248623316992.0
- Affected rows count: 417
- Affected row indices: [0, 36, 155, 234, 283, 426, 548, 617, 630, 683, 719, 761, 904, 1000, 1051, 1246, 1294, 1392, 1485, 1531, 1631, 1779, 1815, 2038, 2071, 2214, 2239, 2292, 2293, 2294, 2295, 2296, 2297, 2298, 2299, 2300, 2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 2312, 2313, 2314, 2315, 2316, 2317, 2318, 2319, 2320, 2321, 2322, 2323, 2324, 2325, 2326, 2327, 2328, 2329, 2330, 2331, 2332, 2333, 2

/Users/henrybaker/repositories/thesis_analysis/scripts/a_data_loading_cleaning.py:611: UserWarning: NB: FLOPs values are NOT constant: [1.69499710e+13 2.02486233e+13 0.00000000e+00 5.26385823e+13]
  if not verify_flops(df):
/Users/henrybaker/repositories/thesis_analysis/scripts/a_data_loading_cleaning.py:621: UserWarning: NB: FLOPs values are NOT constant: [1.69499710e+13 5.26385823e+13]
  verify_flops(df)


In [2]:
cols = df.columns
cols

Index(['config_name', 'experiment_id', 'cycle_id', 'date_time', 'model',
       'num_processes', 'batch_size___fixed_batching', 'decoder_temperature',
       'decoder_top_k', 'decoder_top_p', 'latency_simulation_simulate',
       'latency_simulation_delay_max', 'latency_simulation_delay_min',
       'latency_simulation_simulate_burst', 'latency_simulation_burst_size',
       'latency_simulation_burst_interval', 'fp_precision', 'quantization',
       'load_in_8bit', 'load_in_4bit', 'total_input_tokens',
       'total_generated_tokens', 'date_time', 'total_params',
       'max_input_tokens', 'max_output_tokens', 'number_input_prompts',
       'total_energy_kwh', 'total_energy_joules', 'flops', 'tokens_per_joule',
       'joules_per_token', 'flops_per_joule', 'joules_per_flop',
       'total_inference_time_sec', 'average_latency_ms_per_batch',
       'throughput_queries_per_sec', 'throughput_tokens_per_sec',
       'gpu_utilization_percent_0', 'gpu_utilization_percent_1',
       'gpu_util

In [11]:
import numpy as np
import pandas as pd
import patsy
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1) Copy and create your derived columns
df2 = df.copy()

# 1a) Numeric batch size
df2['batch_size_numeric'] = pd.to_numeric(
    df2['batch_size___fixed_batching'], errors='coerce'
)

# 1b) Collapse precision levels into one ordinal variable
df2['precision_bits'] = 32
df2.loc[df2['fp_precision']=='torch.float16',     'precision_bits'] = 16
df2.loc[df2['load_in_8bit']         == True,      'precision_bits'] = 8
df2.loc[df2['load_in_4bit']         == True,      'precision_bits'] = 4

# 1c) Collapse decoder mode + param
df2['decoding_mode']   = df2['decoder_config_decoding_mode'].astype('category')
df2['sampling_param']  = 0.0
mask_k = df2['decoding_mode']=='top_k'
mask_p = df2['decoding_mode']=='top_p'
df2.loc[mask_k, 'sampling_param'] = df2.loc[mask_k, 'decoder_top_k']
df2.loc[mask_p, 'sampling_param'] = df2.loc[mask_p, 'decoder_top_p']

# 1d) Drop the old columns you no longer need
df2 = df2.drop(columns=[
    'fp_precision','quantization','load_in_8bit','load_in_4bit',
    'decoder_config_decoding_mode','decoder_top_k','decoder_top_p',
    'batch_size___fixed_batching'
])

# 2) Define which columns must be present (no NAs) for the VIF check
keep = [
    'energy_per_token_kwh','batch_size_numeric','num_processes',
    'precision_bits','decoder_temperature','sampling_param',
    'decoding_mode'
]

# 3) Drop rows where any of these is missing
df3 = df2.dropna(subset=keep)

# 4) Build and inspect the design matrix
formula = (
    "energy_per_token_kwh ~ batch_size_numeric + num_processes + "
    "precision_bits + decoder_temperature + sampling_param + "
    "C(decoding_mode)"
)
y, X = patsy.dmatrices(formula, df3, return_type='dataframe')

print("Shape:", X.shape)
print("Rank :", np.linalg.matrix_rank(X.values))
print("Cond :", np.linalg.cond(X.values))

# 5) Compute VIFs
vif = pd.DataFrame({
    'variable': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
}).sort_values('VIF', ascending=False)

print(vif)


Shape: (1687, 8)
Rank : 5
Cond : 1.97255664858682e+20
                    variable       VIF
1  C(decoding_mode)[T.top_k]  5.188534
2  C(decoding_mode)[T.top_p]  4.715084
7             sampling_param  1.470129
6        decoder_temperature  1.000182
0                  Intercept  0.000000
3         batch_size_numeric  0.000000
4              num_processes  0.000000
5             precision_bits  0.000000


/Users/henrybaker/miniconda3/envs/thesis/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


In [12]:
from sklearn.preprocessing import StandardScaler

# columns to standardize
num_cols = ['batch_size_numeric','num_processes','precision_bits',
            'decoder_temperature','sampling_param']

X_std = X.copy()
X_std[num_cols] = StandardScaler().fit_transform(X[num_cols])

print("Post‐scaling condition number:", np.linalg.cond(X_std.values))


Post‐scaling condition number: 3.6859591916646165e+34


In [13]:
import numpy as np

# full SVD
U, s, Vt = np.linalg.svd(X.values, full_matrices=False)

# identify “zero” singular values
tol = 1e-8
null_idxs = np.where(s < tol)[0]

# extract the corresponding right‐singular vectors
nullspace = Vt[null_idxs, :]

# display
for i, v in enumerate(nullspace, 1):
    print(f"\nNull‐space vector #{i}:")
    for col, coeff in zip(X.columns, v):
        if abs(coeff) > 1e-6:
            print(f"  {coeff:+.3f} × {col}")



Null‐space vector #1:
  -0.999 × Intercept
  +0.009 × batch_size_numeric
  -0.032 × num_processes
  +0.031 × precision_bits

Null‐space vector #2:
  +0.011 × Intercept
  -0.877 × batch_size_numeric
  -0.148 × num_processes
  +0.457 × precision_bits

Null‐space vector #3:
  -0.035 × Intercept
  -0.182 × batch_size_numeric
  +0.982 × num_processes
  -0.031 × precision_bits


In [14]:
import numpy as np
import patsy
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1) Mean‐center your numeric predictors
for col in ['batch_size_numeric','num_processes','precision_bits']:
    df3[col + '_c'] = df3[col] - df3[col].mean()

# 2) Build the new formula replacing raw with centered
formula_c = (
    "energy_per_token_kwh ~ batch_size_numeric_c + num_processes_c + "
    "precision_bits_c + decoder_temperature + sampling_param + "
    "C(decoding_mode)"
)

# 3) Reconstruct the design matrix
y, Xc = patsy.dmatrices(formula_c, df3, return_type='dataframe')

print("Shape:", Xc.shape)
print("Rank :", np.linalg.matrix_rank(Xc.values))
print("Cond :", np.linalg.cond(Xc.values))

# 4) VIFs (drop intercept before VIF)
Xc_noint = Xc.drop(columns='Intercept')
vif = (
    pd.DataFrame({
        'variable': Xc_noint.columns,
        'VIF': [variance_inflation_factor(Xc_noint.values, i)
                for i in range(Xc_noint.shape[1])]
    })
    .sort_values('VIF', ascending=False)
)
print(vif)


Shape: (1687, 8)
Rank : 5
Cond : 7.668850724557086e+35
                    variable       VIF
0  C(decoding_mode)[T.top_k]  3.058715
5        decoder_temperature  2.909338
6             sampling_param  2.006183
1  C(decoding_mode)[T.top_p]  1.845121
2       batch_size_numeric_c       NaN
3            num_processes_c       NaN
4           precision_bits_c       NaN


/var/folders/9b/zwzkyxhs7h7759872hzp3mz80000gn/T/ipykernel_5872/810911859.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3[col + '_c'] = df3[col] - df3[col].mean()
/var/folders/9b/zwzkyxhs7h7759872hzp3mz80000gn/T/ipykernel_5872/810911859.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3[col + '_c'] = df3[col] - df3[col].mean()
/var/folders/9b/zwzkyxhs7h7759872hzp3mz80000gn/T/ipykernel_5872/810911859.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame